In [33]:
! pip install transformers torch


In [45]:
from transformers import pipeline
import re

classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
from transformers import pipeline
import re
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
nltk.download('punkt')

# get model and tokenizer 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
# def function to deal with one file
def classify_and_save_sentences(input_file, output_file):
    # open file 
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # separate the sentences  to 32 parts for each file
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    fixed_length = max(1, int(num_sentences / 32))
    merged_sentences = [' '.join(sentences[i:i + fixed_length]) for i in range(0, len(sentences), fixed_length)]

    # get the results for each parts(tokenizer is used to Converts each part into tokens suitable for the model.) 
    results = []
    for merged in merged_sentences:
        inputs = tokenizer(merged, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        label = model.config.id2label[predicted_class_id]
        score = torch.nn.functional.softmax(logits, dim=1).max().item()
        results.append({"label": label, "score": score})
    
    # write the results into new_file
    with open(output_file, 'w') as f:
        for sentence, result in zip(merged_sentences, results):
            f.write(f"Sentence: {sentence}\n")
            f.write(f"Prediction: {result['label']} (score: {result['score']})\n\n")

    output_to_csv = output_file.replace('.txt','.csv')
    df = pd.DataFrame(results)
    df.to_csv(output_to_csv, index=False)

    print(f"Results saved to {output_file}")

# test
#input_file = "/Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/BERT_annotations/BERT_annotations/dialog/12 Monkeys_0114746_anno_dialog.txt"
#output_file = "/Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/BERT_annotations/dialog_analyse/12 Monkeys_0114746_anno_dialog.txt"


#lassify_and_save_sentences(input_file, output_file)




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiaozhouye/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [51]:
# make new dir for output 
os.makedirs("/Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64",exist_ok=True)

In [52]:
import pandas as pd
import os
try:
    directory_path = '--input your path'
    files = os.listdir(directory_path)
    #print(len(files))
    for file in files:
        file = os.path.join(directory_path,file)
        classify_sentences(file, '--input your directory pat for output')
except Exception as e:
    print(f"Error occurred: {e}")

Results saved to /Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64/Before Midnight_2209418_anno_dialog.txt
Before Midnight_2209418_anno_dialog.txt is saved
Results saved to /Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64/John Carter_0401729_anno_dialog.txt
John Carter_0401729_anno_dialog.txt is saved
Results saved to /Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64/The Conspirator_0968264_anno_dialog.txt
The Conspirator_0968264_anno_dialog.txt is saved
Results saved to /Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64/The Iron Lady_1007029_anno_dialog.txt
The Iron Lady_1007029_anno_dialog.txt is saved
Results saved to /Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/all_scripts_64/Battle for the Planet of the Apes_0069768_anno_dialog.txt
Battle for th

KeyboardInterrupt: 